In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# save the final model to file
import keras
import numpy as np
from keras.datasets import fashion_mnist
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	# one hot encode target values
	trainY = keras.utils.np_utils.to_categorical(trainY)
	testY = keras.utils.np_utils.to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model(learn=0.001,moment=0.9):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=learn, momentum=moment)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=100, batch_size=64, verbose=1)
	# save model
	model.save('final_model.h5')

# entry point, run the test harness
# run_test_harness()

In [ ]:

def train_one_model(learn=0.001,moment=0.9,bs=64,i=0):
  # load dataset
  trainX, trainY, testX, testY = load_dataset()
  # prepare pixel data
  trainX, testX = prep_pixels(trainX, testX)
  # define model
  model = define_model(learn,moment)
  # fit model
  model.fit(trainX, trainY, epochs=1, batch_size=bs, verbose=1)
  while model.evaluate( trainX, trainY,batch_size=bs)[0]>0.1:
    model.fit(trainX, trainY, epochs=1, batch_size=bs, verbose=1)

  # save model
  model.save('fm_model_'+str(i)+'_learn'+str(learn)+'_batch'+str(bs)+'.h5')
  del model
# train_one_model()

In [ ]:
bss = [64,128]
lrs = [0.003,0.001]

In [ ]:
for bsk in bss:
    for lr in lrs:
        for j in range(5):
            train_one_model(learn=lr,moment=0.9,bs=bsk,i=j)

In [ ]:
from keras import backend as k
tf.compat.v1.disable_eager_execution()

def var_grad(mod,X,y):
    outputTensor = model.output #Or model.layers[index].output
    listOfVariableTensors = model.trainable_weights
    gradients = k.gradients(outputTensor, listOfVariableTensors)
    sess = tf.compat.v1.InteractiveSession()
    sess.run(tf.compat.v1.initialize_all_variables())
    evaluated_gradients = sess.run(gradients,feed_dict={model.input:X})

    print(evaluated_gradients)
    

In [ ]:
from keras.models import load_model
# load dataset
trainX, trainY, testX, testY = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

model = load_model('../input/models/model_0_learn0.001_batch128.h5')
var_grad(model,trainX[:5000],trainY[:5000])
# pacbayes_sigma(model_fn=model,image_tensor=trainX[:5000], label_tensor=trainY[:5000], training_accuracy=model.evaluate(trainX, trainY)[1], target_deviate=0.1,
#     checkpoint_directory='../input/models')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as k

from keras.models import load_model
# load dataset
trainX, trainY, testX, testY = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
def unpack(a):
    b = []
    for i in range(len(a)):
        if type(a[i]) == type(a[0]):
            for j in a[i]:
                b.append(j)
            continue
        if type(a[i]) == type(a[30]):
            b.append(a[i])
            continue
    return b

variances = np.array((2,2,5))
gaps = np.array((2,2,5))
j=0
t=0

for bsk in bss:
    for lr in lrs:
        for i in range(5):
            print('model_'+str(i)+'_learn'+str(lr)+'_batch'+str(bsk)+'.h5')
            model = load_model('../input/models/model_'+str(i)+'_learn'+str(lr)+'_batch'+str(bsk)+'.h5')
            trainX = trainX[:5000]
            outputTensor = model.output #Or model.layers[index].output

            listOfVariableTensors = model.trainable_weights
            gradients = k.gradients(outputTensor, listOfVariableTensors)
            sess = tf.compat.v1.InteractiveSession()
            sess.run(tf.compat.v1.initialize_all_variables())
            evaluated_gradients = sess.run(gradients,feed_dict={model.input:trainX})
            
            a=[item for sublist in evaluated_gradients for item in sublist]
            print(np.var(unpack(unpack(unpack(a)))))

In [ ]:
type(a[0])


In [ ]:
def unpack(a):
    b = []
    for i in range(len(a)):
        if type(a[i]) == type(a[0]):
            for j in a[i]:
                b.append(j)
            continue
        if type(a[i]) == type(a[30]):
            b.append(a[i])
            continue
    return b
for bsk in bss:
    for lr in lrs:
        for j in range(5):
            train_one_model(learn=lr,moment=0.9,bs=bsk,i=j)
print(np.var(unpack(unpack(unpack(a)))))

In [ ]:
variances = np.zeros(shape=(2,2,5))
gaps = np.zeros(shape=(2,2,5))
t=0
for bsk in bss:
    j=0
    for lr in lrs:
        for i in range(5):
            print(i)
            print(j)
            print(t)
            variances[t][j][i] = 0
            gaps[t][j][i] = 0
        j+=1
    t+=1
        

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as k
variances = np.zeros(shape=(2,2,5))
gaps = np.zeros(shape=(2,2,5))
j=0
t=0
for bsk in bss:
    j=0
    for lr in lrs:
        for i in range(5):
            # load dataset
            trainX, trainY, testX, testY = load_dataset()
            # prepare pixel data
            trainX, testX = prep_pixels(trainX, testX)
            print('model_'+str(i)+'_learn'+str(lr)+'_batch'+str(bsk)+'.h5')
            model = load_model('../input/models/model_'+str(i)+'_learn'+str(lr)+'_batch'+str(bsk)+'.h5')
            print(model.evaluate( trainX, trainY,batch_size=128)[1])
            print(model.evaluate( testX, testY,batch_size=128)[1])

            gap = model.evaluate( trainX, trainY,batch_size=128)[1]-model.evaluate( testX, testY,batch_size=128)[1]
            trainX = trainX[:5000]
            outputTensor = model.output #Or model.layers[index].output

            listOfVariableTensors = model.trainable_weights
# listOfVariableTensors = model.trainable_weights[-1]
#or variableTensors = model.trainable_weights[0]
            gradients = k.gradients(outputTensor, listOfVariableTensors)
            sess = tf.compat.v1.InteractiveSession()
            sess.run(tf.compat.v1.initialize_all_variables())
            evaluated_gradients = sess.run(gradients,feed_dict={model.input:trainX})
            
            a=[item for sublist in evaluated_gradients for item in sublist]
            var = np.var(unpack(unpack(unpack(a))))
            variances[t][j][i] = var
            gaps[t][j][i] = gap
            print('var is :' + str(var))
            print('gap is :' + str(gap))
        j+=1
    t+=1

In [ ]:
gaps

In [ ]:
tao = np.zeros(shape=(5))
for i in range(5):
    sumi = 0
    for t in range(2):
        for j in range(2):
            u1 = variances[t][j][i]
            g1 = gaps[t][j][i]

            for p in range(2):
                for d in range(2):
                    if(p==t and d==j):
                        continue
                    u2 = variances[p][d][i]
                    g2 = gaps[p][d][i]
                    sumi+=np.sign(u1-u2)*np.sign(g1-g2)
    tao[i] = (1/(4*3))*sumi

In [ ]:
tao

In [ ]:
np.mean(tao)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
sns.boxplot(data=tao,orient='v',showmeans=True,meanline=True,bootstrap=10000,notch=True)
plt.title('Weight gradients Complexity Measure Tao rank')
plt.xlabel('Tao rank')
plt.savefig('kendal')

In [ ]:
mi=2.0
n=2.0
psbatch = np.zeros(shape=(5))
pslearn = np.zeros(shape=(5))
granu_kendal = np.zeros(shape=(5))
for i in range(5):
    sumi=0
    for j in range(2):
        u1 = variances[0][j][i]
        g1 = gaps[0][j][i]
        u2 = variances[1][j][i]
        g2 = gaps[1][j][i]
        sumi+=np.sign(u1-u2)*np.sign(g1-g2)
    psbatch[i] = (1/mi)*sumi

for i in range(5):
    sumi=0
    for j in range(2):
        u1 = variances[j][0][i]
        g1 = gaps[j][0][i]
        u2 = variances[j][1][i]
        g2 = gaps[j][1][i]
        sumi+=np.sign(u1-u2)*np.sign(g1-g2)
    pslearn[i] = (1/mi)*sumi
granu_kendal = (1/n)*(psbatch+pslearn)
sns.set_theme(style="whitegrid")
sns.boxplot(data=granu_kendal,orient='v',showmeans=True,meanline=True,notch=True)
plt.title('Weight gradients Complexity Measure Granulated Tao rank')
plt.xlabel('Granulated Tao rank')
plt.savefig('Granulatedkendal')

In [ ]:
granu_kendal